In [56]:
from mpa.models import Mpa, mpa_post_save
from django.db.models.signals import post_save, post_delete
from mpa.views import mpas_all_nogeom
import reversion
from reversion.models import Revision
from reversion.models import Version
import datetime
from django.db.models import Q, F, Func

In [57]:
mpaset = mpas_all_nogeom.filter(
    Q(country__icontains='GBR') | Q(sovereign__icontains='GBR')
    | Q(country__icontains='AIA') | Q(country__icontains='BMU') | Q(country__icontains='CYM')
    | Q(country__icontains='FLK') | Q(country__icontains='GIB') | Q(country__icontains='IMN')
    | Q(country__icontains='IOT') | Q(country__icontains='JEY') | Q(country__icontains='MSR')
    | Q(country__icontains='PCN') | Q(country__icontains='SGS') | Q(country__icontains='SHN')
    | Q(country__icontains='TCA') | Q(country__icontains='VGB')
)
mpaset.values('country').distinct()

<QuerySet [{'country': 'BMU'}, {'country': 'MSR'}, {'country': 'GIB'}, {'country': 'IOT'}, {'country': 'GBR'}, {'country': 'VGB'}, {'country': 'TCA'}, {'country': 'SGS'}, {'country': 'FLK'}, {'country': 'AIA'}, {'country': 'JEY'}, {'country': 'IMN'}, {'country': 'PCN'}, {'country': 'SHN'}, {'country': 'CYM'}]>

In [58]:
mpas_all_nogeom.filter(wdpa_id=0).update(wdpa_id=None)
mpas_all_nogeom.filter(wdpa_id=0).count()

0

In [59]:
count = 0
for m in mpaset:
    versions = Version.objects.get_for_object(m)
    created = 'Never'
    if not m.wdpa_id: continue
    try:
        created = versions.first().revision.date_created
        if created > datetime.datetime(2014, 11, 5):
            count = count + 1
            print(count, created, 'count:', len(versions), 'mpa_id:', m.mpa_id, 'wdpa_id:', m.wdpa_id)
    except:
        pass

1 2015-07-01 13:07:36.256517 count: 10 mpa_id: 15066 wdpa_id: 555559203
2 2016-10-22 12:35:01.829706 count: 1 mpa_id: 6580 wdpa_id: 32641
3 2018-10-23 16:59:25.207976 count: 1 mpa_id: 7702510 wdpa_id: 93767
4 2018-10-23 16:52:57.166931 count: 1 mpa_id: 68809149 wdpa_id: 555583067
5 2014-11-05 16:34:11.597338 count: 1 mpa_id: 6615 wdpa_id: 37040
6 2018-10-23 17:00:14.136746 count: 4 mpa_id: 16069 wdpa_id: 555583066
7 2018-10-23 17:00:14.600354 count: 3 mpa_id: 16071 wdpa_id: 555583070
8 2018-10-24 12:44:50.191532 count: 3 mpa_id: 1982 wdpa_id: 62904
9 2018-10-23 16:52:57.500567 count: 1 mpa_id: 68809150 wdpa_id: 555583071
10 2018-10-23 16:58:22.274499 count: 2 mpa_id: 7263 wdpa_id: 478035
11 2018-10-23 16:58:24.383427 count: 2 mpa_id: 7076 wdpa_id: 312930
12 2014-11-05 16:34:11.936952 count: 1 mpa_id: 6628 wdpa_id: 40944
13 2018-10-24 16:36:07.304004 count: 4 mpa_id: 68808365 wdpa_id: 555547601
14 2018-10-24 16:36:14.197694 count: 4 mpa_id: 68808366 wdpa_id: 555547601
15 2018-10-24 16:3

122 2015-07-01 13:19:08.067761 count: 7 mpa_id: 15059 wdpa_id: 555559196
123 2014-11-05 16:34:06.055022 count: 1 mpa_id: 6284 wdpa_id: 14076
124 2014-11-05 16:34:06.871656 count: 1 mpa_id: 6311 wdpa_id: 14225
125 2015-06-02 10:31:57.875735 count: 3 mpa_id: 2858 wdpa_id: 137240
126 2015-05-22 10:25:04.852039 count: 3 mpa_id: 6872 wdpa_id: 166875
127 2014-12-08 12:24:34.560859 count: 1 mpa_id: 7700984 wdpa_id: 12780
128 2015-07-01 13:23:06.765084 count: 6 mpa_id: 15057 wdpa_id: 555559194
129 2014-12-08 12:29:41.901006 count: 1 mpa_id: 7700981 wdpa_id: 12777
130 2014-12-08 12:00:30.219791 count: 1 mpa_id: 7700995 wdpa_id: 12791
131 2015-09-21 14:56:02.609275 count: 2 mpa_id: 1431 wdpa_id: 18960
132 2015-09-24 15:12:50.991702 count: 2 mpa_id: 1513 wdpa_id: 20604
133 2015-06-02 10:25:33.927248 count: 4 mpa_id: 6843 wdpa_id: 134954
134 2015-09-21 14:54:33.512308 count: 3 mpa_id: 1437 wdpa_id: 18969
135 2015-09-21 14:55:05.450207 count: 2 mpa_id: 1435 wdpa_id: 18965
136 2015-07-01 13:16:43.66

244 2014-11-05 16:37:01.134624 count: 2 mpa_id: 1668 wdpa_id: 26607
245 2014-11-05 16:39:25.651279 count: 1 mpa_id: 3604 wdpa_id: 220216
246 2014-11-05 16:39:25.564337 count: 1 mpa_id: 3602 wdpa_id: 220214
247 2014-11-05 16:39:25.279034 count: 1 mpa_id: 3596 wdpa_id: 220208
248 2014-11-05 16:39:25.376893 count: 1 mpa_id: 3598 wdpa_id: 220210
249 2014-11-05 16:39:25.021881 count: 1 mpa_id: 3591 wdpa_id: 220203
250 2014-11-05 16:39:25.215201 count: 1 mpa_id: 3595 wdpa_id: 220207
251 2014-11-05 16:36:28.247815 count: 1 mpa_id: 1005 wdpa_id: 12832
252 2014-11-05 16:36:25.782482 count: 1 mpa_id: 977 wdpa_id: 12766
253 2014-11-05 16:36:23.665558 count: 1 mpa_id: 951 wdpa_id: 12433
254 2015-06-02 13:23:38.102230 count: 5 mpa_id: 2863 wdpa_id: 137999
255 2015-07-01 12:54:56.640744 count: 6 mpa_id: 15073 wdpa_id: 555559210
256 2015-05-22 10:30:24.760159 count: 1 mpa_id: 7706840 wdpa_id: 127895
257 2015-07-01 12:41:47.426744 count: 8 mpa_id: 15080 wdpa_id: 555559217
258 2015-09-21 14:48:22.73878

365 2018-03-01 07:40:34.270663 count: 1 mpa_id: 8803488 wdpa_id: 193572
366 2015-06-03 06:54:30.609253 count: 1 mpa_id: 7702371 wdpa_id: 68276
367 2014-11-05 16:34:16.671382 count: 1 mpa_id: 6951 wdpa_id: 220081
368 2015-01-07 12:48:53.082387 count: 2 mpa_id: 67706282 wdpa_id: 14074
369 2015-01-07 12:57:12.089619 count: 1 mpa_id: 67701415 wdpa_id: 18780
370 2018-07-16 14:00:41.324349 count: 5 mpa_id: 991 wdpa_id: 12787
371 2018-07-16 14:05:22.767125 count: 6 mpa_id: 1003 wdpa_id: 12799
372 2018-07-16 14:02:28.806427 count: 4 mpa_id: 992 wdpa_id: 12788
373 2018-07-16 14:05:34.758084 count: 7 mpa_id: 1001 wdpa_id: 12797
374 2015-04-09 13:31:21.931070 count: 4 mpa_id: 2521 wdpa_id: 95292
375 2018-07-16 14:05:54.181350 count: 13 mpa_id: 3816 wdpa_id: 303905
376 2018-07-16 14:05:59.894557 count: 6 mpa_id: 1004 wdpa_id: 12800
377 2014-11-05 16:36:40.957959 count: 1 mpa_id: 1170 wdpa_id: 13980
378 2016-10-22 12:34:53.282715 count: 1 mpa_id: 6614 wdpa_id: 36066
379 2015-06-02 13:49:37.017615 c

505 2015-07-15 12:16:04.255609 count: 10 mpa_id: 15064 wdpa_id: 555559201
506 2015-05-22 10:13:15.169921 count: 1 mpa_id: 7706945 wdpa_id: 220072
507 2015-05-26 10:09:49.995653 count: 1 mpa_id: 7707270 wdpa_id: 900585
508 2014-11-05 16:37:12.578143 count: 1 mpa_id: 1846 wdpa_id: 32646
509 2014-11-05 16:36:25.906646 count: 1 mpa_id: 979 wdpa_id: 12772
510 2014-11-05 16:38:55.283708 count: 1 mpa_id: 3312 wdpa_id: 166937
511 2014-11-05 16:36:51.183359 count: 1 mpa_id: 1416 wdpa_id: 18781
512 2015-01-07 12:55:10.477881 count: 1 mpa_id: 67701792 wdpa_id: 31288
513 2014-12-08 11:53:32.012335 count: 3 mpa_id: 3823 wdpa_id: 303914
514 2015-06-02 10:37:17.884058 count: 3 mpa_id: 2883 wdpa_id: 140934
515 2015-09-02 12:27:28.432138 count: 3 mpa_id: 5793 wdpa_id: 555536245
516 2015-09-24 15:26:57.360084 count: 2 mpa_id: 1516 wdpa_id: 20619
517 2015-05-22 10:43:12.624518 count: 1 mpa_id: 7702838 wdpa_id: 127890
518 2015-08-11 10:32:06.631900 count: 1 mpa_id: 5791 wdpa_id: 555536241
519 2015-05-22 1

In [61]:
from importlib import reload
from wdpa import merge
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [62]:
remove_log = '/home/mpatlas/workspace/log/removewdpa_GBR_log.json'
wdpa2remove = merge.getRemoveWdpaList(logfile=remove_log)

In [63]:
len(wdpa2remove)

157

In [64]:
rm_mpas = Mpa.objects.filter(wdpa_id__in=wdpa2remove).order_by('country')
len(rm_mpas)

158

In [65]:
for m in rm_mpas:
    print(m.mpa_id, m.wdpa_id, m.country, m.sovereign, m.is_mpa, m.no_take, m.no_take_area, m.name, m.designation_eng)

1716 28132 BMU GBR True Not Reported 0.0 Little Head Park
1687 28083 BMU GBR True Not Reported 0.0 Nelly Island Park
1715 28131 BMU GBR True Not Reported 0.0 Breman Island Park
1717 28133 BMU GBR True Not Reported 0.0 Great Head Park
1701 28115 BMU GBR True Not Reported 0.0 Kindley Field Park
1682 28078 BMU GBR True Not Reported 0.0 Church Bay Park
6131 12770 BMU GBR True Not Reported 0.0 Great Sound Private Reserve
1679 28072 BMU GBR True Not Reported 0.0 Mangrove Bay Park
3615 220228 BMU GBR True All 0.0 Hog Breaker Protected Area
1197 14550 BMU GBR True Not Reported 0.0 Summerhaven Nature Reserve
3593 220205 BMU GBR True All 0.0 Aristo Protected Area
3609 220222 BMU GBR True All 0.0 Lartington Protected Area
1700 28112 BMU GBR True Not Reported 0.0 Coney Island Park
1692 28101 BMU GBR True Not Reported 0.0 Robinson Bay Park
1695 28107 BMU GBR True Not Reported 0.0 Burchall Cove Park
1702 28118 BMU GBR True Not Reported 0.0 Rocky Hill Park
1713 28129 BMU GBR True Not Reported 0.0 Smi

In [66]:
merge.removeMpasByWdpaId(wdpa2remove)

In [67]:
reload(merge)

<module 'wdpa.merge' from '/home/mpatlas/projects/django_mpatlas/wdpa/merge.py'>

In [68]:
wdpa2add = merge.getAddWdpaList()

In [69]:
len(wdpa2add)

1097

In [70]:
wdpapid2add = merge.getAddWdpaPidList(verbose=True)

In [71]:
len(wdpapid2add)

0

In [16]:
# Assign records with zones to be the first zone.  Do not rerun.
# In separate notebook

In [72]:
allpid2add = wdpapid2add + [str(int(i)) for i in wdpa2add]
len(allpid2add)

1097

In [73]:
reload(merge)
addpoint_geolog = '/home/mpatlas/workspace/log/addpoint_GBR_log.json'
addpoly_log = '/home/mpatlas/workspace/log/addpoly_GBR_log.json'
addpoly_geolog = '/home/mpatlas/workspace/log/addpoly_GBR_geo_log.json'

In [74]:
points = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=points, poly=False, logfile=None, geologfile=addpoint_geolog, dryrun=True)

True

In [75]:
# post_save.disconnect(mpa_post_save, sender=Mpa)
polys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2add).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=polys, poly=True, logfile=addpoly_log, geologfile=addpoly_geolog, dryrun=True)
# post_save.connect(mpa_post_save, sender=Mpa)

1/1097 adding/updating wdpa_pid 12207 with mpa_id 927
    PID: 12207 has 9 field diffs  and GEOM diff
2/1097 adding/updating wdpa_pid 12896 with mpa_id 1013
    PID: 12896 has 9 field diffs  and GEOM diff
3/1097 adding/updating wdpa_pid 135080 with mpa_id None
4/1097 adding/updating wdpa_pid 135114 with mpa_id None
5/1097 adding/updating wdpa_pid 135143 with mpa_id None
6/1097 adding/updating wdpa_pid 135189 with mpa_id None
7/1097 adding/updating wdpa_pid 135198 with mpa_id 2845
    PID: 135198 has 10 field diffs  and GEOM diff
8/1097 adding/updating wdpa_pid 135226 with mpa_id None
9/1097 adding/updating wdpa_pid 135289 with mpa_id 2846
    PID: 135289 has 10 field diffs  and GEOM diff
10/1097 adding/updating wdpa_pid 135388 with mpa_id None
11/1097 adding/updating wdpa_pid 135463 with mpa_id None
12/1097 adding/updating wdpa_pid 135478 with mpa_id None
13/1097 adding/updating wdpa_pid 135521 with mpa_id None
14/1097 adding/updating wdpa_pid 135522 with mpa_id None
15/1097 adding/upd

123/1097 adding/updating wdpa_pid 139256 with mpa_id None
124/1097 adding/updating wdpa_pid 139260 with mpa_id None
125/1097 adding/updating wdpa_pid 139265 with mpa_id None
126/1097 adding/updating wdpa_pid 139304 with mpa_id None
127/1097 adding/updating wdpa_pid 139325 with mpa_id None
128/1097 adding/updating wdpa_pid 139327 with mpa_id None
129/1097 adding/updating wdpa_pid 139328 with mpa_id None
130/1097 adding/updating wdpa_pid 139340 with mpa_id None
131/1097 adding/updating wdpa_pid 139354 with mpa_id None
132/1097 adding/updating wdpa_pid 139375 with mpa_id None
133/1097 adding/updating wdpa_pid 139378 with mpa_id None
134/1097 adding/updating wdpa_pid 139379 with mpa_id None
135/1097 adding/updating wdpa_pid 139384 with mpa_id None
136/1097 adding/updating wdpa_pid 139385 with mpa_id None
137/1097 adding/updating wdpa_pid 139390 with mpa_id None
138/1097 adding/updating wdpa_pid 139392 with mpa_id None
139/1097 adding/updating wdpa_pid 139400 with mpa_id None
140/1097 addin

252/1097 adding/updating wdpa_pid 170111 with mpa_id None
253/1097 adding/updating wdpa_pid 170124 with mpa_id None
254/1097 adding/updating wdpa_pid 170127 with mpa_id None
255/1097 adding/updating wdpa_pid 170152 with mpa_id None
256/1097 adding/updating wdpa_pid 170160 with mpa_id None
257/1097 adding/updating wdpa_pid 170161 with mpa_id None
258/1097 adding/updating wdpa_pid 170168 with mpa_id None
259/1097 adding/updating wdpa_pid 170181 with mpa_id None
260/1097 adding/updating wdpa_pid 170210 with mpa_id None
261/1097 adding/updating wdpa_pid 170220 with mpa_id None
262/1097 adding/updating wdpa_pid 170242 with mpa_id 3346
    PID: 170242 has 10 field diffs  and GEOM diff
263/1097 adding/updating wdpa_pid 170243 with mpa_id None
264/1097 adding/updating wdpa_pid 174643 with mpa_id 3389
    PID: 174643 has 9 field diffs  and GEOM diff
265/1097 adding/updating wdpa_pid 174674 with mpa_id None
266/1097 adding/updating wdpa_pid 174675 with mpa_id None
267/1097 adding/updating wdpa_p

380/1097 adding/updating wdpa_pid 555536242 with mpa_id None
381/1097 adding/updating wdpa_pid 555536243 with mpa_id 5792
    PID: 555536243 has 10 field diffs  and GEOM diff
382/1097 adding/updating wdpa_pid 555536244 with mpa_id None
383/1097 adding/updating wdpa_pid 555536245 with mpa_id 5793
    PID: 555536245 has 11 field diffs  and GEOM diff
384/1097 adding/updating wdpa_pid 555536246 with mpa_id None
385/1097 adding/updating wdpa_pid 555536247 with mpa_id None
386/1097 adding/updating wdpa_pid 555536248 with mpa_id None
387/1097 adding/updating wdpa_pid 555536249 with mpa_id None
388/1097 adding/updating wdpa_pid 555536250 with mpa_id None
389/1097 adding/updating wdpa_pid 555536251 with mpa_id None
390/1097 adding/updating wdpa_pid 555541630 with mpa_id None
391/1097 adding/updating wdpa_pid 555541631 with mpa_id None
392/1097 adding/updating wdpa_pid 555541634 with mpa_id None
393/1097 adding/updating wdpa_pid 555541636 with mpa_id None
394/1097 adding/updating wdpa_pid 555541

513/1097 adding/updating wdpa_pid 555556947 with mpa_id None
514/1097 adding/updating wdpa_pid 555556948 with mpa_id None
515/1097 adding/updating wdpa_pid 555556949 with mpa_id None
516/1097 adding/updating wdpa_pid 555556950 with mpa_id None
517/1097 adding/updating wdpa_pid 555556951 with mpa_id None
518/1097 adding/updating wdpa_pid 555556952 with mpa_id None
519/1097 adding/updating wdpa_pid 555556953 with mpa_id None
520/1097 adding/updating wdpa_pid 555556954 with mpa_id None
521/1097 adding/updating wdpa_pid 555556959 with mpa_id None
522/1097 adding/updating wdpa_pid 555556960 with mpa_id None
523/1097 adding/updating wdpa_pid 555556961 with mpa_id None
524/1097 adding/updating wdpa_pid 555556964 with mpa_id None
525/1097 adding/updating wdpa_pid 555556965 with mpa_id None
526/1097 adding/updating wdpa_pid 555556966 with mpa_id None
527/1097 adding/updating wdpa_pid 555556967 with mpa_id None
528/1097 adding/updating wdpa_pid 555556968 with mpa_id None
529/1097 adding/updating

648/1097 adding/updating wdpa_pid 555557180 with mpa_id None
649/1097 adding/updating wdpa_pid 555557182 with mpa_id None
650/1097 adding/updating wdpa_pid 555557183 with mpa_id None
651/1097 adding/updating wdpa_pid 555557184 with mpa_id None
652/1097 adding/updating wdpa_pid 555557186 with mpa_id None
653/1097 adding/updating wdpa_pid 555557187 with mpa_id None
654/1097 adding/updating wdpa_pid 555557188 with mpa_id None
655/1097 adding/updating wdpa_pid 555557189 with mpa_id None
656/1097 adding/updating wdpa_pid 555557195 with mpa_id None
657/1097 adding/updating wdpa_pid 555557197 with mpa_id None
658/1097 adding/updating wdpa_pid 555557198 with mpa_id None
659/1097 adding/updating wdpa_pid 555557199 with mpa_id None
660/1097 adding/updating wdpa_pid 555557200 with mpa_id None
661/1097 adding/updating wdpa_pid 555557201 with mpa_id None
662/1097 adding/updating wdpa_pid 555557202 with mpa_id None
663/1097 adding/updating wdpa_pid 555557203 with mpa_id None
664/1097 adding/updating

783/1097 adding/updating wdpa_pid 555581092 with mpa_id None
784/1097 adding/updating wdpa_pid 555581095 with mpa_id None
785/1097 adding/updating wdpa_pid 555581097 with mpa_id None
786/1097 adding/updating wdpa_pid 555581099 with mpa_id None
787/1097 adding/updating wdpa_pid 555581100 with mpa_id None
788/1097 adding/updating wdpa_pid 555581101 with mpa_id None
789/1097 adding/updating wdpa_pid 555581105 with mpa_id None
790/1097 adding/updating wdpa_pid 555581111 with mpa_id None
791/1097 adding/updating wdpa_pid 555581114 with mpa_id None
792/1097 adding/updating wdpa_pid 555581116 with mpa_id None
793/1097 adding/updating wdpa_pid 555581117 with mpa_id None
794/1097 adding/updating wdpa_pid 555581122 with mpa_id None
795/1097 adding/updating wdpa_pid 555581124 with mpa_id None
796/1097 adding/updating wdpa_pid 555581132 with mpa_id None
797/1097 adding/updating wdpa_pid 555581136 with mpa_id None
798/1097 adding/updating wdpa_pid 555581141 with mpa_id None
799/1097 adding/updating

918/1097 adding/updating wdpa_pid 555583027 with mpa_id None
919/1097 adding/updating wdpa_pid 555583028 with mpa_id None
920/1097 adding/updating wdpa_pid 555583029 with mpa_id None
921/1097 adding/updating wdpa_pid 555583030 with mpa_id None
922/1097 adding/updating wdpa_pid 555583031 with mpa_id None
923/1097 adding/updating wdpa_pid 555583032 with mpa_id None
924/1097 adding/updating wdpa_pid 555583033 with mpa_id None
925/1097 adding/updating wdpa_pid 555583034 with mpa_id None
926/1097 adding/updating wdpa_pid 555583035 with mpa_id None
927/1097 adding/updating wdpa_pid 555583036 with mpa_id None
928/1097 adding/updating wdpa_pid 555583037 with mpa_id None
929/1097 adding/updating wdpa_pid 555583038 with mpa_id None
930/1097 adding/updating wdpa_pid 555583039 with mpa_id None
931/1097 adding/updating wdpa_pid 555583040 with mpa_id None
932/1097 adding/updating wdpa_pid 555583041 with mpa_id None
933/1097 adding/updating wdpa_pid 555583042 with mpa_id None
934/1097 adding/updating

1052/1097 adding/updating wdpa_pid 555624422 with mpa_id None
1053/1097 adding/updating wdpa_pid 555624423 with mpa_id None
1054/1097 adding/updating wdpa_pid 555624425 with mpa_id None
1055/1097 adding/updating wdpa_pid 555624426 with mpa_id None
1056/1097 adding/updating wdpa_pid 555624427 with mpa_id None
1057/1097 adding/updating wdpa_pid 555624428 with mpa_id None
1058/1097 adding/updating wdpa_pid 555624429 with mpa_id None
1059/1097 adding/updating wdpa_pid 555624430 with mpa_id None
1060/1097 adding/updating wdpa_pid 555624431 with mpa_id None
1061/1097 adding/updating wdpa_pid 555624432 with mpa_id None
1062/1097 adding/updating wdpa_pid 555624433 with mpa_id None
1063/1097 adding/updating wdpa_pid 555624434 with mpa_id None
1064/1097 adding/updating wdpa_pid 555624435 with mpa_id None
1065/1097 adding/updating wdpa_pid 555624436 with mpa_id None
1066/1097 adding/updating wdpa_pid 555624437 with mpa_id None
1067/1097 adding/updating wdpa_pid 555624438 with mpa_id None
1068/109

True

In [76]:
reload(merge)
updatepoint_geolog = '/home/mpatlas/workspace/log/updatepoint_GBR_log.json'
updatepoly_log = '/home/mpatlas/workspace/log/updatepoly_GBR_log.json'
updatepoly_geolog = '/home/mpatlas/workspace/log/updatepoly_GBR_geo_log.json'

In [77]:
wdpa_filter = (
    Q(iso3__icontains='GBR') | Q(parent_iso3__icontains='GBR')
    | Q(iso3__icontains='AIA') | Q(iso3__icontains='BMU') | Q(iso3__icontains='CYM')
    | Q(iso3__icontains='FLK') | Q(iso3__icontains='GIB') | Q(iso3__icontains='IMN')
    | Q(iso3__icontains='IOT') | Q(iso3__icontains='JEY') | Q(iso3__icontains='MSR')
    | Q(iso3__icontains='PCN') | Q(iso3__icontains='SGS') | Q(iso3__icontains='SHN')
    | Q(iso3__icontains='TCA') | Q(iso3__icontains='VGB')
)
polys = Wdpa2018Poly.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
points = Wdpa2018Point.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('1','2')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
allpid2update = list(set(list(polys) + list(points)))

polys_nonmarine = Wdpa2018Poly.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('0')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
points_nonmarine = Wdpa2018Point.objects.exclude(wdpa_pid__in=allpid2add).filter(wdpa_filter).filter(marine__in=('0')).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)
allpid2update_nonmarine = list(set(list(polys_nonmarine) + list(points_nonmarine)))

allpid2update_nonmarine_inmpatlas = mpaset.filter(wdpa_pid__in=allpid2update_nonmarine).order_by('wdpa_pid').values_list('wdpa_pid', flat=True)

allpid2update_marine012 = allpid2update + list(allpid2update_nonmarine_inmpatlas)

In [78]:
print('marine:', len(allpid2update), 'nonmarine:', len(allpid2update_nonmarine), 'all:', len(allpid2update_marine012))
print('POINTS --', 
      'marine:', Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2update).count(),
      'nonmarine:', Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2update_nonmarine).count()
)
print('POLYGONS --', 
      'marine:', Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2update).count(),
      'nonmarine:', Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2update_nonmarine).count(),
)
print('MPATLAS Records --', 
      'marine:', mpaset.filter(wdpa_pid__in=allpid2update).count(),
      'nonmarine:', mpaset.filter(wdpa_pid__in=allpid2update_nonmarine).count(),
)

marine: 229 nonmarine: 10710 all: 419
POINTS -- marine: 12 nonmarine: 4
POLYGONS -- marine: 217 nonmarine: 10706
MPATLAS Records -- marine: 335 nonmarine: 190


In [79]:
# All marine=1,2 sites and only marine=0 that exist in MPAtlas
updatepoints = Wdpa2018Point.objects.filter(wdpa_pid__in=allpid2update_marine012).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=updatepoints, poly=False, logfile=None, geologfile=updatepoint_geolog, dryrun=True)

1/13 adding/updating wdpa_pid 13979 with mpa_id 6275
    PID: 13979 has 1 field diffs 
ERROR processing mpa 6275:  'NoneType' object has no attribute 'hexewkb'
2/13 adding/updating wdpa_pid 14074 with mpa_id 67706282
    PID: 14074 has 3 field diffs  and GEOM diff
ERROR processing mpa 67706282:  'NoneType' object has no attribute 'hexewkb'
3/13 adding/updating wdpa_pid 14074 with mpa_id 6282
    PID: 14074 has 1 field diffs 
ERROR processing mpa 6282:  'NoneType' object has no attribute 'hexewkb'
4/13 adding/updating wdpa_pid 18100 with mpa_id 6423
    PID: 18100 has 1 field diffs 
ERROR processing mpa 6423:  'NoneType' object has no attribute 'hexewkb'
5/13 adding/updating wdpa_pid 2126 with mpa_id 5862
    PID: 2126 has 3 field diffs 
ERROR processing mpa 5862:  'NoneType' object has no attribute 'hexewkb'
6/13 adding/updating wdpa_pid 312930 with mpa_id 7076
ERROR processing mpa 7076:  'NoneType' object has no attribute 'hexewkb'
7/13 adding/updating wdpa_pid 313116 with mpa_id 7078

True

In [80]:
# All marine=1,2 sites and only marine=0 that exist in MPAtlas
updatepolys = Wdpa2018Poly.objects.filter(wdpa_pid__in=allpid2update_marine012).order_by('wdpa_pid')
merge.updateMpasFromWdpaQueryset(qs=updatepolys, poly=True, logfile=updatepoly_log, geologfile=updatepoly_geolog, dryrun=True)

1/512 adding/updating wdpa_pid 100837 with mpa_id 2609
    PID: 100837 has 6 field diffs  and GEOM diff
2/512 adding/updating wdpa_pid 100837 with mpa_id 7702609
    PID: 100837 has 10 field diffs  and GEOM diff
3/512 adding/updating wdpa_pid 10557 with mpa_id 834
    PID: 10557 has 8 field diffs  and GEOM diff
4/512 adding/updating wdpa_pid 10559 with mpa_id 835
    PID: 10559 has 8 field diffs  and GEOM diff
5/512 adding/updating wdpa_pid 10941 with mpa_id 859
    PID: 10941 has 9 field diffs  and GEOM diff
6/512 adding/updating wdpa_pid 10942 with mpa_id 860
    PID: 10942 has 8 field diffs  and GEOM diff
7/512 adding/updating wdpa_pid 10948 with mpa_id 861
    PID: 10948 has 9 field diffs  and GEOM diff
8/512 adding/updating wdpa_pid 10962 with mpa_id 862
    PID: 10962 has 8 field diffs  and GEOM diff
9/512 adding/updating wdpa_pid 10973 with mpa_id 863
    PID: 10973 has 8 field diffs  and GEOM diff
10/512 adding/updating wdpa_pid 10976 with mpa_id 864
    PID: 10976 has 8 field 

81/512 adding/updating wdpa_pid 12837 with mpa_id 6139
    PID: 12837 has 8 field diffs 
82/512 adding/updating wdpa_pid 12838 with mpa_id 6140
    PID: 12838 has 7 field diffs 
83/512 adding/updating wdpa_pid 12839 with mpa_id 6141
    PID: 12839 has 8 field diffs 
84/512 adding/updating wdpa_pid 12840 with mpa_id 6142
    PID: 12840 has 8 field diffs 
85/512 adding/updating wdpa_pid 12841 with mpa_id 6143
    PID: 12841 has 9 field diffs 
86/512 adding/updating wdpa_pid 12902 with mpa_id 67706150
    PID: 12902 has 10 field diffs 
87/512 adding/updating wdpa_pid 12902 with mpa_id 6150
    PID: 12902 has 8 field diffs 
88/512 adding/updating wdpa_pid 134954 with mpa_id 6843
    PID: 134954 has 6 field diffs  and GEOM diff
89/512 adding/updating wdpa_pid 134954 with mpa_id 7706843
    PID: 134954 has 10 field diffs 
90/512 adding/updating wdpa_pid 134955 with mpa_id 6844
    PID: 134955 has 6 field diffs  and GEOM diff
91/512 adding/updating wdpa_pid 134955 with mpa_id 7706844
    PID:

163/512 adding/updating wdpa_pid 18783 with mpa_id 1417
    PID: 18783 has 4 field diffs  and GEOM diff
164/512 adding/updating wdpa_pid 18784 with mpa_id 67701418
    PID: 18784 has 7 field diffs  and GEOM diff
165/512 adding/updating wdpa_pid 18784 with mpa_id 1418
    PID: 18784 has 3 field diffs  and GEOM diff
166/512 adding/updating wdpa_pid 18786 with mpa_id 1419
    PID: 18786 has 3 field diffs  and GEOM diff
167/512 adding/updating wdpa_pid 18786 with mpa_id 67701419
    PID: 18786 has 7 field diffs  and GEOM diff
168/512 adding/updating wdpa_pid 18953 with mpa_id 1424
    PID: 18953 has 8 field diffs  and GEOM diff
169/512 adding/updating wdpa_pid 18954 with mpa_id 1425
    PID: 18954 has 8 field diffs  and GEOM diff
170/512 adding/updating wdpa_pid 18955 with mpa_id 1426
    PID: 18955 has 8 field diffs  and GEOM diff
171/512 adding/updating wdpa_pid 18956 with mpa_id 1427
    PID: 18956 has 9 field diffs  and GEOM diff
172/512 adding/updating wdpa_pid 18957 with mpa_id 1428


243/512 adding/updating wdpa_pid 220077 with mpa_id 7706949
    PID: 220077 has 11 field diffs 
244/512 adding/updating wdpa_pid 220079 with mpa_id 7706950
    PID: 220079 has 10 field diffs 
245/512 adding/updating wdpa_pid 220079 with mpa_id 6950
    PID: 220079 has 6 field diffs  and GEOM diff
246/512 adding/updating wdpa_pid 220081 with mpa_id 6951
    PID: 220081 has 7 field diffs 
247/512 adding/updating wdpa_pid 220082 with mpa_id 7706952
    PID: 220082 has 12 field diffs 
248/512 adding/updating wdpa_pid 220082 with mpa_id 6952
    PID: 220082 has 7 field diffs  and GEOM diff
249/512 adding/updating wdpa_pid 220083 with mpa_id 6953
    PID: 220083 has 6 field diffs 
250/512 adding/updating wdpa_pid 220084 with mpa_id 3585
    PID: 220084 has 6 field diffs  and GEOM diff
251/512 adding/updating wdpa_pid 220084 with mpa_id 7703585
    PID: 220084 has 9 field diffs  and GEOM diff
252/512 adding/updating wdpa_pid 220086 with mpa_id 3586
    PID: 220086 has 6 field diffs  and GEOM 

324/512 adding/updating wdpa_pid 36093 with mpa_id 67701871
    PID: 36093 has 10 field diffs  and GEOM diff
325/512 adding/updating wdpa_pid 36094 with mpa_id 1872
    PID: 36094 has 3 field diffs  and GEOM diff
326/512 adding/updating wdpa_pid 36094 with mpa_id 67701872
    PID: 36094 has 10 field diffs  and GEOM diff
327/512 adding/updating wdpa_pid 36095 with mpa_id 67701873
    PID: 36095 has 9 field diffs  and GEOM diff
328/512 adding/updating wdpa_pid 36095 with mpa_id 1873
    PID: 36095 has 3 field diffs  and GEOM diff
329/512 adding/updating wdpa_pid 37040 with mpa_id 6615
    PID: 37040 has 5 field diffs 
330/512 adding/updating wdpa_pid 40934 with mpa_id 6621
    PID: 40934 has 4 field diffs 
331/512 adding/updating wdpa_pid 40935 with mpa_id 6622
    PID: 40935 has 5 field diffs 
332/512 adding/updating wdpa_pid 40938 with mpa_id 6624
    PID: 40938 has 5 field diffs 
333/512 adding/updating wdpa_pid 40940 with mpa_id 6625
    PID: 40940 has 5 field diffs 
334/512 adding/u

402/512 adding/updating wdpa_pid 62905 with mpa_id 1983
    PID: 62905 has 9 field diffs  and GEOM diff
403/512 adding/updating wdpa_pid 62907 with mpa_id 6673
    PID: 62907 has 3 field diffs 
404/512 adding/updating wdpa_pid 62908 with mpa_id 6674
    PID: 62908 has 3 field diffs 
405/512 adding/updating wdpa_pid 62909 with mpa_id 6675
    PID: 62909 has 3 field diffs 
406/512 adding/updating wdpa_pid 62912 with mpa_id 6676
    PID: 62912 has 3 field diffs 
407/512 adding/updating wdpa_pid 62914 with mpa_id 6677
    PID: 62914 has 3 field diffs 
408/512 adding/updating wdpa_pid 62915 with mpa_id 6678
    PID: 62915 has 4 field diffs 
409/512 adding/updating wdpa_pid 62916 with mpa_id 6679
    PID: 62916 has 3 field diffs 
410/512 adding/updating wdpa_pid 62917 with mpa_id 6680
    PID: 62917 has 4 field diffs 
411/512 adding/updating wdpa_pid 62920 with mpa_id 6681
    PID: 62920 has 4 field diffs 
412/512 adding/updating wdpa_pid 62921 with mpa_id 6682
    PID: 62921 has 3 field dif

481/512 adding/updating wdpa_pid 95292 with mpa_id 2521
    PID: 95292 has 6 field diffs  and GEOM diff
Carto Error for mpa_id 2521: ['Quota exceeded by 984KB']
482/512 adding/updating wdpa_pid 95293 with mpa_id 2522
    PID: 95293 has 7 field diffs  and GEOM diff
Carto Error for mpa_id 2522: ['Quota exceeded by 984KB']
483/512 adding/updating wdpa_pid 95293 with mpa_id 7702522
    PID: 95293 has 12 field diffs  and GEOM diff
Carto Error for mpa_id 7702522: ['Quota exceeded by 984KB']
484/512 adding/updating wdpa_pid 95296 with mpa_id 2523
    PID: 95296 has 6 field diffs  and GEOM diff
Carto Error for mpa_id 2523: ['Quota exceeded by 984KB']
485/512 adding/updating wdpa_pid 95296 with mpa_id 7702523
    PID: 95296 has 11 field diffs  and GEOM diff
Carto Error for mpa_id 7702523: ['Quota exceeded by 984KB']
486/512 adding/updating wdpa_pid 95297 with mpa_id 2524
    PID: 95297 has 7 field diffs  and GEOM diff
Carto Error for mpa_id 2524: ['Quota exceeded by 984KB']
487/512 adding/updat

True

In [81]:
mpaset.filter(wdpa_pid__isnull=False).update(
    no_take_wdpa=F('no_take'), no_take_area_wdpa=F('no_take_area')
)

1623

In [82]:
nt = ('All', 'Part', 'None')
ntupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    (~Q(no_take_wdpa__in=('All')) & Q(no_take_mpatlas__in=('All')))
    | (~Q(no_take_wdpa__in=('All', 'Part')) & Q(no_take_mpatlas__in=('All', 'Part')))
    | (Q(no_take_wdpa__in=('Not Applicable')) & Q(no_take_mpatlas__in=('All', 'Part', 'None')))
)
ntupdate.update(no_take=F('no_take_mpatlas'))

5

In [83]:
ntareaupdate = mpaset.filter(wdpa_pid__isnull=False).filter(
    Q(no_take_area_mpatlas__isnull=False) &
    (
        Q(no_take_area_wdpa__isnull=True)
        | Q(no_take_area_mpatlas__gt=F('no_take_area_wdpa'))
    )
)
ntareaupdate.update(no_take_area=F('no_take_area_mpatlas'))

2

In [84]:
ntchanges = ntupdate | ntareaupdate
for n in ntchanges:
    print(n.mpa_id, n.wdpa_id, n.wdpa_pid, n.name, n.designation, n.designation_eng, n.no_take_wdpa, n.no_take_mpatlas, n.no_take_area_wdpa, n.no_take_area_mpatlas)

1198 14553 14553 Meagre Bay Pond Animal Sanctuary Animal Sanctuary Animal Sanctuary Not Applicable All 0.0 0.0
7701198 14553 14553 Meagre Bay Pond Animal Sanctuary Animal Sanctuary Animal Sanctuary Not Applicable All 0.0 None
67701872 36094 36094 Columbus Landfall Marine National Park National Park Not Reported All 0.0 None
1199 14554 14554 Colliers Bay Pond Animal Sanctuary Animal Sanctuary Animal Sanctuary Not Applicable All 0.0 0.0
15067 555559204 555559204 Medway Estuary Marine Conservation Zone Marine Conservation Zone Not Reported Not Reported 0.0 12.1
1201 14557 14557 Booby Pond Nature Reserve Booby Pond Nature Reserve Booby Pond Nature Reserve Not Applicable All 0.0 1.0


In [85]:
ntchanges = ntupdate | ntareaupdate

import csv

with open('/home/mpatlas/workspace/log/notakechange_GBR_log.csv', 'w', newline='') as csvfile:
    fieldnames = ['mpa_id', 'wdpa_id', 'wdpa_pid', 'name', 'designation', 'designation_eng', 'no_take_wdpa', 'no_take_mpatlas', 'no_take_area_wdpa', 'no_take_area_mpatlas']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in ntchanges.values(*fieldnames):
        writer.writerow(row)

In [86]:
for n in ntchanges:
    try:
        w2018 = Wdpa2018Poly.objects.get(wdpa_pid=n.wdpa_pid)
    except:
        w2018 = Wdpa2018Point.objects.get(wdpa_pid=n.wdpa_pid)
    try:
        w2014 = Wdpa2014Polygon.objects.filter(wdpa_pid=F('wdpa_pid'))[0]
    except:
        try:
            w2014 = Wdpa2014Point.objects.filter(wdpa_pid=F('wdpa_pid'))[0]
        except:
            try:
                w2014 = Wdpa2014Polygon.objects.filter(wdpa_id=F('wdpa_id'))[0]
            except:
                w2014 = Wdpa2014Point.objects.filter(wdpa_id=F('wdpa_id'))[0]
    if (n.no_take == w2014.no_take) or (n.no_take_area == w2014.no_tk_area):
        print(w2018.name, w2018.desig, n.no_take_mpatlas, w2018.no_take, w2014.no_take, n.no_take_area_mpatlas, w2018.no_tk_area, w2014.no_tk_area)

Meagre Bay Pond Animal Sanctuary Animal Sanctuary All Not Applicable Not Applicable 0.0 0.0 0.0
Meagre Bay Pond Animal Sanctuary Animal Sanctuary All Not Applicable Not Applicable None 0.0 0.0
Columbus Landfall Marine National Park All Not Reported Not Applicable None 0.0 0.0
Colliers Bay Pond Animal Sanctuary Animal Sanctuary All Not Applicable Not Applicable 0.0 0.0 0.0


In [87]:
ntchanges2 = ntupdate | ntareaupdate
ntchanges.count()

6

In [95]:
mpaset = mpas_all_nogeom.filter(
    Q(country__icontains='GBR') | Q(sovereign__icontains='GBR')
    | Q(country__icontains='AIA') | Q(country__icontains='BMU') | Q(country__icontains='CYM')
    | Q(country__icontains='FLK') | Q(country__icontains='GIB') | Q(country__icontains='IMN')
    | Q(country__icontains='IOT') | Q(country__icontains='JEY') | Q(country__icontains='MSR')
    | Q(country__icontains='PCN') | Q(country__icontains='SGS') | Q(country__icontains='SHN')
    | Q(country__icontains='TCA') | Q(country__icontains='VGB')
)

from mpatlas.utils import cartompa
cartompa.purgeCartoMpas()

[188,
 195,
 498,
 500,
 504,
 512,
 513,
 514,
 515,
 995,
 1429,
 1431,
 1434,
 1435,
 1436,
 1437,
 1438,
 1440,
 1445,
 1503,
 1505,
 1508,
 1509,
 1510,
 1511,
 1513,
 1851,
 1872,
 2362,
 2363,
 2364,
 2365,
 2366,
 2367,
 2369,
 2370,
 2371,
 2375,
 2378,
 2510,
 2517,
 2518,
 2519,
 2522,
 2524,
 3582,
 3819,
 3820,
 3823,
 3824,
 6871,
 6884,
 6935,
 6936,
 6938,
 6942,
 6943,
 6945,
 6946,
 6948,
 6949,
 6952,
 6955,
 6956,
 6957,
 7270,
 7283,
 7299,
 9280,
 9397,
 9398,
 9399,
 9400,
 9401,
 9402,
 9403,
 9405,
 9406,
 9407,
 9408,
 9409,
 9410,
 9411,
 9412,
 9413,
 9414,
 9415,
 9416,
 9417,
 9418,
 9419,
 9420,
 9421,
 9422,
 15058,
 15083,
 7700994,
 7700999,
 7701851,
 7702510,
 7702516,
 7702520,
 7702521,
 7703821,
 7703822,
 7703825,
 7703826,
 7705349,
 7709259]

In [94]:
cartompa.updateAllMpas(mpas=mpaset)

Processing 10 of 1987 mpa records at a time
Records [0 - 9]
Records [10 - 19]
Records [20 - 29]
Records [30 - 39]
ERROR processing mpa 68807503:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807503
Records [40 - 49]
ERROR processing mpa 68807520:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807520
ERROR processing mpa 68807504:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807504
Records [50 - 59]
Records [60 - 69]
ERROR processing mpa 68807500:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807500
ERROR processing mpa 68807495:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 68807495
ERROR processing mpa 60009586:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 60009586
ERROR processing mpa 60009547:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 60009547
ERROR processing mpa 60009546:  'NoneType' object has no attribute 'hexewkb'
Skipping Mpa 60009546
ERROR processing mpa 60009495:  'NoneTyp

KeyboardInterrupt: 